In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model


In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders()

In [12]:
#for x in train_loader:
#    print(x)
#    break

# model

In [13]:
#import pretrainedmodels

In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    """
    EfficientNet B0-B8.
    Args:
        cfg (CfgNode): configs
    """
    def __init__(self, cfg):
        super(BengaliNet, self).__init__()
        model_name = cfg.MODEL_NAME
        pretrained = cfg.PRETRAINED
        input_channels = cfg.IN_CHANNELS
        pool_type = cfg.POOL_TYPE
        drop_connect_rate = cfg.DROP_CONNECT
        self.drop_rate = cfg.DROPOUT
        cls_head = cfg.CLS_HEAD
        num_total_classes = cfg.NUM_GRAPHEME_CLASSES + cfg.NUM_VOWEL_CLASSES + cfg.NUM_CONSONANT_CLASSES \
            + cfg.NUM_WORD_CLASSES

        backbone = timm.create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=input_channels,
            drop_connect_rate=drop_connect_rate,
        )
        self.conv_stem = backbone.conv_stem
        self.bn1 = backbone.bn1
        self.act1 = backbone.act1
        ### Original blocks ###
        for i in range(len((backbone.blocks))):
            setattr(self, "block{}".format(str(i)), backbone.blocks[i])
        self.conv_head = backbone.conv_head
        self.bn2 = backbone.bn2
        self.act2 = backbone.act2
        self.aux_block5 = backbone.blocks[5]
        self.aux_num_features = self.block5[-1].bn3.num_features
        self.aux_head4 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn4 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act4 = Swish()
        self.aux_head5 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn5 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act5 = Swish()
        self.global_pool = SelectAdaptivePool2d(pool_type=pool_type)
        self.num_features = backbone.num_features * self.global_pool.feat_mult()
        assert cls_head == 'linear'
        if cls_head == "linear":
            ### Baseline head ###
            self.fc = nn.Linear(self.num_features, num_total_classes)            
            self.aux_fc1 = nn.Linear(self.aux_num_features*4, num_total_classes)
            self.aux_fc2 = nn.Linear(self.aux_num_features*4, num_total_classes)
            
            for fc in [self.fc, self.aux_fc1, self.aux_fc2]:
                nn.init.zeros_(fc.bias.data)
        elif cls_head == "norm_softmax":
            ### NormSoftmax ###
            self.grapheme_fc = NormSoftmax(self.num_features, num_grapheme_classes)
            self.consonant_fc = NormSoftmax(self.num_features, num_consonant_classes)
            self.vowel_fc = NormSoftmax(self.num_features, num_vowel_classes)
        # Replace with Mish activation
        if cfg.MODEL_ACTIVATION == "mish":
            convert_swish_to_mish(self)
        del backbone

    def _features(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x); b4 = x
        x = self.block5(x); b4 = self.aux_block5(b4); b5 = x
        x = self.block6(x)
        x = self.conv_head(x); b4 = self.aux_head4(b4); b5 = self.aux_head5(b5)
        x = self.bn2(x); b4 = self.bn4(b4); b5 = self.bn5(b5)
        x = self.act2(x); b4 = self.act4(b4); b5 = self.act5(b5)
        return b4, b5, x

    def forward(self, x):
        x = F.interpolate(x, size=(380, 380), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)

        # _, _, x = self._features(x)
        b4, b5, x = self._features(x)
        x = self.global_pool(x); b4 = self.global_pool(b4); b5 = self.global_pool(b5)
        x = torch.flatten(x, 1); b4 = torch.flatten(b4, 1); b5 = torch.flatten(b5, 1)
        if self.drop_rate > 0.:
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        logits = self.fc(x)
        
        aux_logits1 = self.aux_fc1(b4)
        aux_logits2 = self.aux_fc2(b5)
        
        return logits, aux_logits1, aux_logits2

In [16]:
MODEL_DIR = './model4-ckps'
def create_model(cfg):
    model = BengaliNet(cfg)
    model_file = os.path.join(MODEL_DIR, cfg.MODEL_NAME, cfg.CKP_NAME)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [17]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [18]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [19]:
round(1/9, 6)

0.111111

In [20]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(y[:, 0], preds0, average='macro')
    recall_vowel = sklearn.metrics.recall_score(y[:, 1], preds1, average='macro')
    recall_consonant = sklearn.metrics.recall_score(y[:, 2], preds2, average='macro')
    recall_word = sklearn.metrics.recall_score(y[:, 3], preds3, average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    old_recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    old_recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    old_recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    old_recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    old_scores = [old_recall_grapheme, old_recall_vowel, old_recall_consonant]
    old_final_recall_score = np.average(old_scores, weights=[2, 1, 1])

    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)  
    
    metrics['old_recall'] = round(old_final_recall_score, 6)
    metrics['old_recall_grapheme'] = round(old_recall_grapheme, 6)
    
    return metrics

In [21]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()

In [22]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, _, _ = model(x)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [23]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [24]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [25]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [26]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [27]:
def rand_bbox_new(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [28]:
np.random.random()

0.8270489315337932

In [29]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [30]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [31]:
import time
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0
    bg = time.time()

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        if r < 0.3:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            #img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.7:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, targets)
            loss_aux1 = criterion(outputs_aux1, targets)
            loss_aux2 = criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            orig_img, targets = mixup(orig_img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(orig_img)
            loss_primary = mixup_criterion(outputs, targets)
            loss_aux1 = mixup_criterion(outputs_aux1, targets)
            loss_aux2 = mixup_criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} | {:.2f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1), (time.time() - bg) / 60), end='')


In [32]:
import copy
best_metrics = 0.
best_metrics_swa = 0.


def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()

def validate_and_save_swa(model, model_file, val_loader, save=False):
    global best_metrics_swa
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics_swa:
        best_metrics_swa = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()


def train(args):
    model, model_file = create_model(cfg)
    model = model.cuda()

    swa_cfg = copy.deepcopy(cfg)
    swa_cfg.CKP_NAME = cfg.CKP_NAME + '_swa'
    swa_model, swa_model_file = create_model(swa_cfg)
    swa_model = swa_model.cuda()

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    #[model, swa_model], optimizer = amp.initialize(
    #    [model, swa_model], optimizer, opt_level="O2",verbosity=0, keep_batchnorm_fp32=True)
    
    #opt_level="O2", keep_batchnorm_fp32=True
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)

    swa_model_loaded = False
    if os.path.exists(swa_model_file):
        swa_model_loaded = True
        validate_and_save_swa(swa_model, swa_model_file, val_loader, save=False)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.start_epoch, args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                if not swa_model_loaded:
                    copy_model(swa_model, model)
                #swa_n = 0
                swa_n = args.swa_n
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save_swa(swa_model, swa_model_file, val_loader, save=True)

        #args.base_lr = 2e-4
        #args.num_epochs = 40
        #args.warmup_epochs = 1

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

In [33]:
cfg = Namespace()
cfg.MODEL_NAME = 'tf_efficientnet_b4'
cfg.PRETRAINED = True
cfg.IN_CHANNELS = 1
cfg.POOL_TYPE = 'avg'
cfg.CLS_HEAD = 'linear'
cfg.MODEL_ACTIVATION = 'swish'
cfg.DROP_CONNECT = 0.2
cfg.DROPOUT= 0.
cfg.NUM_WORD_CLASSES = 1295
cfg.NUM_GRAPHEME_CLASSES = 168
cfg.NUM_VOWEL_CLASSES = 11
cfg.NUM_CONSONANT_CLASSES = 7
cfg.CKP_NAME = 'model4_eb4_fold2_380.pth'

In [34]:
#model, model_file = create_model(cfg)
#model(torch.randn(2,1,137,236))[0].size()

In [35]:
args = Namespace()

args.base_lr = 2e-4
args.num_epochs = 60
args.start_epoch = 0
args.warmup_epochs = 5
args.num_cycles = 100
args.batch_size = 128
args.val_batch_size = 512
args.st_epochs = 5

args.swa_start = 2
args.swa_freq = 3
args.swa_n = 3

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=2)

(200840, 6)
(200840, 32332)
(160678, 6) (40162, 6)


In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998693, 'recall_grapheme': 0.99817, 'recall_vowel': 0.999095, 'recall_consonant': 0.999337, 'recall_word': 0.997772, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999178, 'acc_consonant': 0.999303, 'acc_word': 0.997809, 'old_recall': 0.998635, 'old_recall_grapheme': 0.998031, 'loss_grapheme': 0.010964, 'loss_vowel': 0.005378, 'loss_consonant': 0.003544, 'loss_word': 0.011642}

val: {'recall': 0.998742, 'recall_grapheme': 0.998106, 'recall_vowel': 0.999245, 'recall_consonant': 0.999511, 'recall_word': 0.998126, 'acc_grapheme': 0.998282, 'acc_vowel': 0.999328, 'acc_consonant': 0.999452, 'acc_word': 0.998182, 'old_recall': 0.998894, 'old_recall_grapheme': 0.9984, 'loss

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998635, 'recall_grapheme': 0.998031, 'recall_vowel': 0.999094, 'recall_consonant': 0.999381, 'recall_word': 0.99789, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999178, 'acc_consonant': 0.999303, 'acc_word': 0.997809, 'loss_grapheme': 0.010964, 'loss_vowel': 0.005378, 'loss_consonant': 0.003544, 'loss_word': 0.011642}

val: {'recall': 0.998894, 'recall_grapheme': 0.9984, 'recall_vowel': 0.999307, 'recall_consonant': 0.999469, 'recall_word': 0.998212, 'acc_grapheme': 0.998282, 'acc_vowel': 0.999328, 'acc_consonant': 0.999452, 'acc_word': 0.998182, 'loss_grapheme': 0.008724, 'loss_vowel': 0.004496, 'loss_consonant': 0.002706, 'loss_word': 0.009823}
CYCLE: 1
    0 | 

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998635, 'recall_grapheme': 0.998031, 'recall_vowel': 0.999094, 'recall_consonant': 0.999381, 'recall_word': 0.99789, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999178, 'acc_consonant': 0.999303, 'acc_word': 0.997809, 'loss_grapheme': 0.010964, 'loss_vowel': 0.005378, 'loss_consonant': 0.003544, 'loss_word': 0.011642}

val: {'recall': 0.998894, 'recall_grapheme': 0.9984, 'recall_vowel': 0.999307, 'recall_consonant': 0.999469, 'recall_word': 0.998212, 'acc_grapheme': 0.998282, 'acc_vowel': 0.999328, 'acc_consonant': 0.999452, 'acc_word': 0.998182, 'loss_grapheme': 0.008724, 'loss_vowel': 0.004496, 'loss_consonant': 0.002706, 'loss_word': 0.009823}
CYCLE: 1
    0 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998153, 'recall_grapheme': 0.997244, 'recall_vowel': 0.998818, 'recall_consonant': 0.999304, 'recall_word': 0.997294, 'acc_grapheme': 0.99741, 'acc_vowel': 0.999054, 'acc_consonant': 0.999278, 'acc_word': 0.997236, 'loss_grapheme': 0.012261, 'loss_vowel': 0.005854, 'loss_consonant': 0.003568, 'loss_word': 0.013006}
SWA>>>:


100%|██████████| 1255/1255 [07:08<00:00,  2.91it/s]



val: {'recall': 0.998664, 'recall_grapheme': 0.99798, 'recall_vowel': 0.99932, 'recall_consonant': 0.999377, 'recall_word': 0.998081, 'acc_grapheme': 0.998182, 'acc_vowel': 0.999353, 'acc_consonant': 0.999402, 'acc_word': 0.998033, 'loss_grapheme': 0.00864, 'loss_vowel': 0.004549, 'loss_consonant': 0.002757, 'loss_word': 0.009867}
    3 | 0.000152 | 134400/160678 | 9.8910 | 5.1330 | 11.73 ||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998635, 'recall_grapheme': 0.998031, 'recall_vowel': 0.999094, 'recall_consonant': 0.999381, 'recall_word': 0.99789, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999178, 'acc_consonant': 0.999303, 'acc_word': 0.997809, 'loss_grapheme': 0.010964, 'loss_vowel': 0.005378, 'loss_consonant': 0.003544, 'loss_word': 0.011642}

val: {'recall': 0.99877, 'recall_grapheme': 0.998213, 'recall_vowel': 0.999307, 'recall_consonant': 0.999345, 'recall_word': 0.998201, 'acc_grapheme': 0.998232, 'acc_vowel': 0.999328, 'acc_consonant': 0.999427, 'acc_word': 0.998182, 'loss_grapheme': 0.008726, 'loss_vowel': 0.004444, 'loss_consonant': 0.002708, 'loss_word': 0.009676}
CYCLE: 1
    0 |

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998362, 'recall_grapheme': 0.99774, 'recall_vowel': 0.998866, 'recall_consonant': 0.999104, 'recall_word': 0.997574, 'acc_grapheme': 0.997759, 'acc_vowel': 0.999104, 'acc_consonant': 0.999278, 'acc_word': 0.99751, 'loss_grapheme': 0.012019, 'loss_vowel': 0.005669, 'loss_consonant': 0.003796, 'loss_word': 0.012675}
SWA>>>:


100%|██████████| 1255/1255 [07:06<00:00,  2.79it/s]



val: {'recall': 0.998835, 'recall_grapheme': 0.99834, 'recall_vowel': 0.999223, 'recall_consonant': 0.999435, 'recall_word': 0.998161, 'acc_grapheme': 0.998207, 'acc_vowel': 0.999303, 'acc_consonant': 0.999427, 'acc_word': 0.998133, 'loss_grapheme': 0.008634, 'loss_vowel': 0.004441, 'loss_consonant': 0.002688, 'loss_word': 0.009663}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa
    3 | 0.000099 | 160640/160678 | 5.7569 | 5.0590 | 14.22 ||
val: {'recall': 0.998404, 'recall_grapheme': 0.997578, 'recall_vowel': 0.999013, 'recall_consonant': 0.999446, 'recall_word': 0.997654, 'acc_grapheme': 0.997635, 'acc_vowel': 0.999129, 'acc_consonant': 0.999353, 'acc_word': 0.997585, 'loss_grapheme': 0.010987, 'loss_vowel': 0.005414, 'loss_consonant': 0.003538, 'loss_word': 0.011809}
    4 | 0.000123 | 160640/160678 | 0.1739 | 5.1097 | 14.33 ||
val: {'recall': 0.998144, 'recall_grapheme': 0.997111, 'recall_vowel': 0.999064, 'recall_consonant': 0.999288, 'recall_word': 0

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998477, 'recall_grapheme': 0.997627, 'recall_vowel': 0.999149, 'recall_consonant': 0.999504, 'recall_word': 0.997792, 'acc_grapheme': 0.997859, 'acc_vowel': 0.999104, 'acc_consonant': 0.999378, 'acc_word': 0.997734, 'loss_grapheme': 0.010453, 'loss_vowel': 0.005473, 'loss_consonant': 0.003032, 'loss_word': 0.011557}
SWA>>>:


100%|██████████| 1255/1255 [07:11<00:00,  2.69it/s]



val: {'recall': 0.998724, 'recall_grapheme': 0.998133, 'recall_vowel': 0.999292, 'recall_consonant': 0.999339, 'recall_word': 0.998104, 'acc_grapheme': 0.998282, 'acc_vowel': 0.999303, 'acc_consonant': 0.999402, 'acc_word': 0.998083, 'loss_grapheme': 0.008308, 'loss_vowel': 0.004388, 'loss_consonant': 0.002688, 'loss_word': 0.009364}
   45 | 0.000032 | 157312/160678 | 10.9290 | 5.0613 | 14.27 |

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998635, 'recall_grapheme': 0.998031, 'recall_vowel': 0.999094, 'recall_consonant': 0.999381, 'recall_word': 0.99789, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999178, 'acc_consonant': 0.999303, 'acc_word': 0.997809, 'loss_grapheme': 0.010964, 'loss_vowel': 0.005378, 'loss_consonant': 0.003544, 'loss_word': 0.011642}

val: {'recall': 0.998699, 'recall_grapheme': 0.998145, 'recall_vowel': 0.999195, 'recall_consonant': 0.999311, 'recall_word': 0.998223, 'acc_grapheme': 0.998157, 'acc_vowel': 0.999278, 'acc_consonant': 0.999402, 'acc_word': 0.998207, 'loss_grapheme': 0.00876, 'loss_vowel': 0.004544, 'loss_consonant': 0.002715, 'loss_word': 0.00971}
CYCLE: 1
    0 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997923, 'recall_grapheme': 0.997141, 'recall_vowel': 0.998443, 'recall_consonant': 0.998969, 'recall_word': 0.996917, 'acc_grapheme': 0.996788, 'acc_vowel': 0.998705, 'acc_consonant': 0.998979, 'acc_word': 0.996838, 'loss_grapheme': 0.015744, 'loss_vowel': 0.006867, 'loss_consonant': 0.004946, 'loss_word': 0.015373}
SWA>>>:


100%|██████████| 1255/1255 [07:08<00:00,  2.82it/s]



val: {'recall': 0.99877, 'recall_grapheme': 0.998213, 'recall_vowel': 0.999307, 'recall_consonant': 0.999345, 'recall_word': 0.998201, 'acc_grapheme': 0.998232, 'acc_vowel': 0.999328, 'acc_consonant': 0.999427, 'acc_word': 0.998182, 'loss_grapheme': 0.008726, 'loss_vowel': 0.004444, 'loss_consonant': 0.002708, 'loss_word': 0.009676}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa
    2 | 0.000195 | 160640/160678 | 6.1217 | 5.1876 | 13.95 ||

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998107, 'recall_grapheme': 0.997296, 'recall_vowel': 0.998884, 'recall_consonant': 0.99895, 'recall_word': 0.997125, 'acc_grapheme': 0.997311, 'acc_vowel': 0.998929, 'acc_consonant': 0.999079, 'acc_word': 0.997037, 'loss_grapheme': 0.01306, 'loss_vowel': 0.006658, 'loss_consonant': 0.00459, 'loss_word': 0.013936}
SWA>>>:


100%|██████████| 1255/1255 [07:04<00:00,  2.89it/s]



val: {'recall': 0.998659, 'recall_grapheme': 0.997983, 'recall_vowel': 0.99917, 'recall_consonant': 0.999499, 'recall_word': 0.998235, 'acc_grapheme': 0.998232, 'acc_vowel': 0.999278, 'acc_consonant': 0.999477, 'acc_word': 0.998207, 'loss_grapheme': 0.008521, 'loss_vowel': 0.004618, 'loss_consonant': 0.002588, 'loss_word': 0.009628}
    6 | 0.000175 | 150528/160678 | 0.1704 | 5.2703 | 13.41 ||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998333, 'recall_grapheme': 0.99753, 'recall_vowel': 0.99891, 'recall_consonant': 0.999362, 'recall_word': 0.997679, 'acc_grapheme': 0.997859, 'acc_vowel': 0.999104, 'acc_consonant': 0.999378, 'acc_word': 0.997585, 'loss_grapheme': 0.011018, 'loss_vowel': 0.005305, 'loss_consonant': 0.003185, 'loss_word': 0.011873}

val: {'recall': 0.998693, 'recall_grapheme': 0.998173, 'recall_vowel': 0.999111, 'recall_consonant': 0.999317, 'recall_word': 0.998115, 'acc_grapheme': 0.998133, 'acc_vowel': 0.999228, 'acc_consonant': 0.999427, 'acc_word': 0.998083, 'loss_grapheme': 0.009206, 'loss_vowel': 0.004678, 'loss_consonant': 0.002865, 'loss_word': 0.010286}
CYCLE: 1
   14 |

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997335, 'recall_grapheme': 0.995933, 'recall_vowel': 0.998453, 'recall_consonant': 0.999021, 'recall_word': 0.996675, 'acc_grapheme': 0.996489, 'acc_vowel': 0.998655, 'acc_consonant': 0.999004, 'acc_word': 0.996614, 'loss_grapheme': 0.017132, 'loss_vowel': 0.007924, 'loss_consonant': 0.004922, 'loss_word': 0.016749}
SWA>>>:


100%|██████████| 1255/1255 [07:00<00:00,  2.89it/s]



val: {'recall': 0.998679, 'recall_grapheme': 0.998159, 'recall_vowel': 0.999113, 'recall_consonant': 0.999283, 'recall_word': 0.998124, 'acc_grapheme': 0.998182, 'acc_vowel': 0.999203, 'acc_consonant': 0.999402, 'acc_word': 0.998108, 'loss_grapheme': 0.009091, 'loss_vowel': 0.00466, 'loss_consonant': 0.002802, 'loss_word': 0.010135}
   15 | 0.000230 | 160640/160678 | 0.7302 | 5.2954 | 14.26 ||
val: {'recall': 0.997845, 'recall_grapheme': 0.996834, 'recall_vowel': 0.998844, 'recall_consonant': 0.998867, 'recall_word': 0.996814, 'acc_grapheme': 0.996713, 'acc_vowel': 0.99883, 'acc_consonant': 0.999004, 'acc_word': 0.996763, 'loss_grapheme': 0.015115, 'loss_vowel': 0.006485, 'loss_consonant': 0.004953, 'loss_word': 0.015424}
   16 | 0.000222 | 160640/160678 | 2.3889 | 5.3610 | 14.23 ||
val: {'recall': 0.997374, 'recall_grapheme': 0.996045, 'recall_vowel': 0.998492, 'recall_consonant': 0.998916, 'recall_word': 0.996365, 'acc_grapheme': 0.996464, 'acc_vowel': 0.998382, 'acc_consonant': 0.9

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998461, 'recall_grapheme': 0.997827, 'recall_vowel': 0.998904, 'recall_consonant': 0.999286, 'recall_word': 0.997761, 'acc_grapheme': 0.997734, 'acc_vowel': 0.999029, 'acc_consonant': 0.999278, 'acc_word': 0.997684, 'loss_grapheme': 0.011131, 'loss_vowel': 0.005547, 'loss_consonant': 0.003633, 'loss_word': 0.011709}
SWA>>>:


100%|██████████| 1255/1255 [07:06<00:00,  2.82it/s]



val: {'recall': 0.998684, 'recall_grapheme': 0.998103, 'recall_vowel': 0.999184, 'recall_consonant': 0.999345, 'recall_word': 0.998161, 'acc_grapheme': 0.998133, 'acc_vowel': 0.999253, 'acc_consonant': 0.999427, 'acc_word': 0.998133, 'loss_grapheme': 0.00873, 'loss_vowel': 0.004539, 'loss_consonant': 0.002699, 'loss_word': 0.009694}
   45 | 0.000005 | 160640/160678 | 7.3381 | 5.2069 | 14.30 ||
val: {'recall': 0.998233, 'recall_grapheme': 0.99732, 'recall_vowel': 0.998974, 'recall_consonant': 0.999317, 'recall_word': 0.99744, 'acc_grapheme': 0.997386, 'acc_vowel': 0.999029, 'acc_consonant': 0.999253, 'acc_word': 0.997336, 'loss_grapheme': 0.01287, 'loss_vowel': 0.00583, 'loss_consonant': 0.003914, 'loss_word': 0.013365}
   46 | 0.000005 | 003840/160678 | 6.0223 | 5.7321 | 0.39 ||

KeyboardInterrupt: 

In [38]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa...

val: {'recall': 0.998268, 'recall_grapheme': 0.997585, 'recall_vowel': 0.998839, 'recall_consonant': 0.999061, 'recall_word': 0.997331, 'acc_grapheme': 0.99746, 'acc_vowel': 0.999104, 'acc_consonant': 0.999228, 'acc_word': 0.997286, 'loss_grapheme': 0.012183, 'loss_vowel': 0.005558, 'loss_consonant': 0.0036, 'loss_word': 0.012152}

val: {'recall': 0.99858, 'recall_grapheme': 0.997991, 'recall_vowel': 0.999056, 'recall_consonant': 0.99928, 'recall_word': 0.997955, 'acc_grapheme': 0.998008, 'acc_vowel': 0.999228, 'acc_consonant': 0.999402, 'acc_word': 0.997933, 'loss_grapheme': 0.009319, 'loss_vowel': 0.004726, 'loss_consonant': 0.002979, 'loss_word': 0.010399}
CYCLE: 1
   14 | 0.

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997629, 'recall_grapheme': 0.996466, 'recall_vowel': 0.998614, 'recall_consonant': 0.998971, 'recall_word': 0.996339, 'acc_grapheme': 0.996564, 'acc_vowel': 0.998755, 'acc_consonant': 0.999029, 'acc_word': 0.99629, 'loss_grapheme': 0.015955, 'loss_vowel': 0.006842, 'loss_consonant': 0.004633, 'loss_word': 0.015611}
SWA>>>:


100%|██████████| 1255/1255 [07:19<00:00,  2.87it/s]



val: {'recall': 0.998625, 'recall_grapheme': 0.998087, 'recall_vowel': 0.999044, 'recall_consonant': 0.99928, 'recall_word': 0.997929, 'acc_grapheme': 0.998033, 'acc_vowel': 0.999228, 'acc_consonant': 0.999402, 'acc_word': 0.997908, 'loss_grapheme': 0.009307, 'loss_vowel': 0.004739, 'loss_consonant': 0.002963, 'loss_word': 0.010409}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa
   15 | 0.000230 | 160640/160678 | 13.9007 | 5.8293 | 14.33 |
val: {'recall': 0.997549, 'recall_grapheme': 0.996796, 'recall_vowel': 0.998846, 'recall_consonant': 0.997757, 'recall_word': 0.996716, 'acc_grapheme': 0.996788, 'acc_vowel': 0.998929, 'acc_consonant': 0.998805, 'acc_word': 0.996688, 'loss_grapheme': 0.017405, 'loss_vowel': 0.009103, 'loss_consonant': 0.007679, 'loss_word': 0.01575}
   16 | 0.000222 | 160640/160678 | 3.7559 | 5.4858 | 14.41 ||
val: {'recall': 0.996871, 'recall_grapheme': 0.996295, 'recall_vowel': 0.998393, 'recall_consonant': 0.9965, 'recall_word': 0.99

...

100%|██████████| 1255/1255 [07:30<00:00,  2.78it/s]



val: {'recall': 0.998665, 'recall_grapheme': 0.998165, 'recall_vowel': 0.999055, 'recall_consonant': 0.999275, 'recall_word': 0.998098, 'acc_grapheme': 0.998257, 'acc_vowel': 0.999228, 'acc_consonant': 0.999378, 'acc_word': 0.998083, 'loss_grapheme': 0.009232, 'loss_vowel': 0.004671, 'loss_consonant': 0.002901, 'loss_word': 0.010244}
   33 | 0.000070 | 160640/160678 | 10.0970 | 5.3108 | 14.49 |
val: {'recall': 0.998132, 'recall_grapheme': 0.997275, 'recall_vowel': 0.99888, 'recall_consonant': 0.999098, 'recall_word': 0.997563, 'acc_grapheme': 0.997684, 'acc_vowel': 0.999153, 'acc_consonant': 0.999253, 'acc_word': 0.99751, 'loss_grapheme': 0.012275, 'loss_vowel': 0.005423, 'loss_consonant': 0.003501, 'loss_word': 0.012497}
   34 | 0.000062 | 160640/160678 | 3.5376 | 5.3721 | 14.50 ||
val: {'recall': 0.997322, 'recall_grapheme': 0.996334, 'recall_vowel': 0.998822, 'recall_consonant': 0.997796, 'recall_word': 0.997072, 'acc_grapheme': 0.996913, 'acc_vowel': 0.998929, 'acc_consonant': 0.9

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 1255/1255 [07:29<00:00,  2.83it/s]



val: {'recall': 0.998642, 'recall_grapheme': 0.998135, 'recall_vowel': 0.999055, 'recall_consonant': 0.999244, 'recall_word': 0.998099, 'acc_grapheme': 0.998207, 'acc_vowel': 0.999228, 'acc_consonant': 0.999353, 'acc_word': 0.998083, 'loss_grapheme': 0.009189, 'loss_vowel': 0.004649, 'loss_consonant': 0.002898, 'loss_word': 0.010201}
   39 | 0.000031 | 086784/160678 | 6.6328 | 5.1872 | 7.83 ||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: False

val: {'recall': 0.997777, 'recall_grapheme': 0.997203, 'recall_vowel': 0.9987, 'recall_consonant': 0.998002, 'recall_word': 0.997111, 'acc_grapheme': 0.997311, 'acc_vowel': 0.99888, 'acc_consonant': 0.999104, 'acc_word': 0.997037, 'loss_grapheme': 0.014067, 'loss_vowel': 0.006681, 'loss_consonant': 0.004011, 'loss_word': 0.014713}
CYCLE: 1
    8 | 0.000277 | 160640/160678 | 17.6224 | 5.4163 | 14.52 |
val: {'recall': 0.997021, 'recall_grapheme': 0.995488, 'recall_vowel': 0.998185, 'recall_consonant': 0.998923, 'recall_word': 0.995881, 'acc_grapheme': 0.996091, 'acc_vowel': 0.998531, 'acc_consonant': 0.998805, 'acc_word': 0.995817, 'loss_grapheme': 0.018216, 'loss_vowel': 0.008646, 'loss_consonant': 0.005926, 'loss_word': 0.018805}
    9 | 0.000271 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.996452, 'recall_grapheme': 0.995173, 'recall_vowel': 0.99805, 'recall_consonant': 0.997411, 'recall_word': 0.995446, 'acc_grapheme': 0.995692, 'acc_vowel': 0.998282, 'acc_consonant': 0.998357, 'acc_word': 0.995443, 'loss_grapheme': 0.020047, 'loss_vowel': 0.008777, 'loss_consonant': 0.007031, 'loss_word': 0.020246}
SWA>>>:


100%|██████████| 1255/1255 [07:22<00:00,  2.74it/s]



val: {'recall': 0.996948, 'recall_grapheme': 0.995878, 'recall_vowel': 0.998365, 'recall_consonant': 0.997671, 'recall_word': 0.996224, 'acc_grapheme': 0.996365, 'acc_vowel': 0.998606, 'acc_consonant': 0.99873, 'acc_word': 0.996166, 'loss_grapheme': 0.014388, 'loss_vowel': 0.006843, 'loss_consonant': 0.005331, 'loss_word': 0.015355}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa
   10 | 0.000266 | 160640/160678 | 0.7395 | 5.6629 | 14.55 ||
val: {'recall': 0.996777, 'recall_grapheme': 0.995017, 'recall_vowel': 0.998473, 'recall_consonant': 0.9986, 'recall_word': 0.995581, 'acc_grapheme': 0.995817, 'acc_vowel': 0.998456, 'acc_consonant': 0.998531, 'acc_word': 0.995543, 'loss_grapheme': 0.01995, 'loss_vowel': 0.008123, 'loss_consonant': 0.006372, 'loss_word': 0.018806}
   11 | 0.000259 | 160640/160678 | 3.3898 | 5.8648 | 14.48 ||

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998145, 'recall_grapheme': 0.997263, 'recall_vowel': 0.998961, 'recall_consonant': 0.999095, 'recall_word': 0.99734, 'acc_grapheme': 0.99746, 'acc_vowel': 0.999153, 'acc_consonant': 0.999253, 'acc_word': 0.997286, 'loss_grapheme': 0.011741, 'loss_vowel': 0.005311, 'loss_consonant': 0.003502, 'loss_word': 0.011915}
SWA>>>:


100%|██████████| 1255/1255 [07:30<00:00,  2.70it/s]



val: {'recall': 0.998554, 'recall_grapheme': 0.997807, 'recall_vowel': 0.999103, 'recall_consonant': 0.999499, 'recall_word': 0.997983, 'acc_grapheme': 0.997933, 'acc_vowel': 0.999253, 'acc_consonant': 0.999477, 'acc_word': 0.997933, 'loss_grapheme': 0.009159, 'loss_vowel': 0.004458, 'loss_consonant': 0.002894, 'loss_word': 0.01011}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa
CYCLE: 2
    8 | 0.000277 | 160640/160678 | 6.5008 | 5.4948 | 14.68 ||
val: {'recall': 0.996787, 'recall_grapheme': 0.996248, 'recall_vowel': 0.998255, 'recall_consonant': 0.996396, 'recall_word': 0.996018, 'acc_grapheme': 0.99624, 'acc_vowel': 0.998282, 'acc_consonant': 0.998481, 'acc_word': 0.995966, 'loss_grapheme': 0.018003, 'loss_vowel': 0.008212, 'loss_consonant': 0.005905, 'loss_word': 0.018469}
    9 | 0.000273 | 104064/160678 | 6.6273 | 5.6662 | 9.48 |||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: False

val: {'recall': 0.996864, 'recall_grapheme': 0.995341, 'recall_vowel': 0.998716, 'recall_consonant': 0.998059, 'recall_word': 0.995527, 'acc_grapheme': 0.995643, 'acc_vowel': 0.998456, 'acc_consonant': 0.998531, 'acc_word': 0.995543, 'loss_grapheme': 0.021051, 'loss_vowel': 0.009045, 'loss_consonant': 0.00687, 'loss_word': 0.019851}
CYCLE: 1
    0 | 0.000399 | 160640/160678 | 0.5148 | 5.8375 | 13.83 ||
val: {'recall': 0.994827, 'recall_grapheme': 0.993436, 'recall_vowel': 0.997847, 'recall_consonant': 0.994591, 'recall_word': 0.99367, 'acc_grapheme': 0.993501, 'acc_vowel': 0.997834, 'acc_consonant': 0.997784, 'acc_word': 0.993576, 'loss_grapheme': 0.027582, 'loss_vowel': 0.012216, 'loss_consonant': 0.01028, 'loss_word': 0.026025}
    1 | 0.000398 | 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   20 | 0.000184 | 160640/160678 | 3.5539 | 5.8566 | 14.42 ||
val: {'recall': 0.996555, 'recall_grapheme': 0.994989, 'recall_vowel': 0.998411, 'recall_consonant': 0.99783, 'recall_word': 0.996139, 'acc_grapheme': 0.996439, 'acc_vowel': 0.998357, 'acc_consonant': 0.99883, 'acc_word': 0.996141, 'loss_grapheme': 0.016667, 'loss_vowel': 0.007622, 'loss_consonant': 0.005235, 'loss_word': 0.016536}
   21 | 0.000169 | 160640/160678 | 7.9928 | 5.4756 | 14.50 ||
val: {'recall': 0.996231, 'recall_grapheme': 0.994384, 'recall_vowel': 0.998098, 'recall_consonant': 0.998057, 'recall_word': 0.995292, 'acc_grapheme': 0.995518, 'acc_vowel': 0.998282, 'acc_consonant': 0.998631, 'acc_word': 0.995319, 'loss_grapheme': 0.019995, 'loss_vowel': 0.008366, 'loss_consonant': 0.005858, 'loss_word': 0.019362}
   22 | 0.000153 | 160640/160678 | 0.1657 | 5.4465 | 14.47 ||
val: {'recall': 0.996016, 'recall_grapheme': 0.994153, 'recall_vowel': 0.997745, 'recall_consonant': 0.998014, 'recall_word': 0.995117, 'acc_gra

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   28 | 0.000070 | 160640/160678 | 7.1218 | 5.6596 | 14.44 ||
val: {'recall': 0.997226, 'recall_grapheme': 0.995506, 'recall_vowel': 0.998533, 'recall_consonant': 0.999359, 'recall_word': 0.996388, 'acc_grapheme': 0.996315, 'acc_vowel': 0.998456, 'acc_consonant': 0.999178, 'acc_word': 0.99639, 'loss_grapheme': 0.01523, 'loss_vowel': 0.007205, 'loss_consonant': 0.004184, 'loss_word': 0.015197}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth
   29 | 0.000059 | 160640/160678 | 0.9150 | 5.2730 | 14.51 ||
val: {'recall': 0.997157, 'recall_grapheme': 0.995557, 'recall_vowel': 0.998705, 'recall_consonant': 0.998809, 'recall_word': 0.99652, 'acc_grapheme': 0.996539, 'acc_vowel': 0.998581, 'acc_consonant': 0.999129, 'acc_word': 0.996539, 'loss_grapheme': 0.014689, 'loss_vowel': 0.007272, 'loss_consonant': 0.004466, 'loss_word': 0.014702}
   30 | 0.000048 | 160640/160678 | 11.7399 | 5.5561 | 14.44 |
val: {'recall': 0.996746, 'recall_grapheme': 0.995126, 'recall_vowel': 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   13 | 0.000145 | 160640/160678 | 3.6052 | 5.3858 | 14.55 ||
val: {'recall': 0.9972, 'recall_grapheme': 0.996244, 'recall_vowel': 0.998612, 'recall_consonant': 0.9977, 'recall_word': 0.996528, 'acc_grapheme': 0.99639, 'acc_vowel': 0.998556, 'acc_consonant': 0.99868, 'acc_word': 0.996439, 'loss_grapheme': 0.01712, 'loss_vowel': 0.006885, 'loss_consonant': 0.005825, 'loss_word': 0.016743}
   14 | 0.000138 | 160640/160678 | 2.5435 | 5.4727 | 14.51 ||
val: {'recall': 0.997059, 'recall_grapheme': 0.995488, 'recall_vowel': 0.998863, 'recall_consonant': 0.998399, 'recall_word': 0.996368, 'acc_grapheme': 0.996464, 'acc_vowel': 0.99878, 'acc_consonant': 0.998855, 'acc_word': 0.99634, 'loss_grapheme': 0.017036, 'loss_vowel': 0.006625, 'loss_consonant': 0.005677, 'loss_word': 0.0171}
   15 | 0.000131 | 160640/160678 | 5.5252 | 5.3911 | 14.55 ||
val: {'recall': 0.997457, 'recall_grapheme': 0.996516, 'recall_vowel': 0.998866, 'recall_consonant': 0.997931, 'recall_word': 0.997129, 'acc_grapheme': 0

   33 | 0.000011 | 160640/160678 | 6.0712 | 5.0062 | 14.53 ||
val: {'recall': 0.997484, 'recall_grapheme': 0.996719, 'recall_vowel': 0.998857, 'recall_consonant': 0.997638, 'recall_word': 0.996982, 'acc_grapheme': 0.997037, 'acc_vowel': 0.999054, 'acc_consonant': 0.999228, 'acc_word': 0.996962, 'loss_grapheme': 0.014909, 'loss_vowel': 0.006764, 'loss_consonant': 0.004052, 'loss_word': 0.015049}
   34 | 0.000008 | 160640/160678 | 12.4042 | 5.1806 | 14.50 |
val: {'recall': 0.997746, 'recall_grapheme': 0.997055, 'recall_vowel': 0.99883, 'recall_consonant': 0.998047, 'recall_word': 0.997351, 'acc_grapheme': 0.99741, 'acc_vowel': 0.999004, 'acc_consonant': 0.999178, 'acc_word': 0.997311, 'loss_grapheme': 0.012888, 'loss_vowel': 0.006307, 'loss_consonant': 0.003611, 'loss_word': 0.013399}
   35 | 0.000005 | 160640/160678 | 12.4352 | 5.1254 | 14.57 |

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth_swa, exist: False

val: {'recall': 0.863007, 'recall_grapheme': 0.799473, 'recall_vowel': 0.910495, 'recall_consonant': 0.942587, 'recall_word': 0.792659, 'acc_grapheme': 0.579279, 'acc_vowel': 0.896942, 'acc_consonant': 0.816369, 'acc_word': 0.390718, 'loss_grapheme': 1.777621, 'loss_vowel': 0.442087, 'loss_consonant': 0.433245, 'loss_word': 2.889786}
CYCLE: 1


/home/chec/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    0 | 0.000292 | 160640/160678 | 9.0631 | 11.0421 | 13.91 ||
val: {'recall': 0.990122, 'recall_grapheme': 0.987137, 'recall_vowel': 0.994537, 'recall_consonant': 0.991677, 'recall_word': 0.986548, 'acc_grapheme': 0.987476, 'acc_vowel': 0.994921, 'acc_consonant': 0.995892, 'acc_word': 0.986355, 'loss_grapheme': 0.081153, 'loss_vowel': 0.043929, 'loss_consonant': 0.032938, 'loss_word': 0.071857}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth
    1 | 0.000543 | 160640/160678 | 11.2862 | 7.7721 | 14.02 |
val: {'recall': 0.99057, 'recall_grapheme': 0.985973, 'recall_vowel': 0.994439, 'recall_consonant': 0.995896, 'recall_word': 0.987708, 'acc_grapheme': 0.98765, 'acc_vowel': 0.994921, 'acc_consonant': 0.996116, 'acc_word': 0.987227, 'loss_grapheme': 0.050568, 'loss_vowel': 0.022158, 'loss_consonant': 0.018201, 'loss_word': 0.050209}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold2_380.pth
    2 | 0.000476 | 160640/160678 | 1.3762 | 7.5592 | 14.18 

    9 | 0.000171 | 160640/160678 | 6.6217 | 6.1082 | 14.56 ||
val: {'recall': 0.99469, 'recall_grapheme': 0.993247, 'recall_vowel': 0.997689, 'recall_consonant': 0.994578, 'recall_word': 0.993979, 'acc_grapheme': 0.993974, 'acc_vowel': 0.997958, 'acc_consonant': 0.997834, 'acc_word': 0.993974, 'loss_grapheme': 0.028306, 'loss_vowel': 0.012581, 'loss_consonant': 0.009692, 'loss_word': 0.026933}
   10 | 0.000165 | 160640/160678 | 13.6986 | 5.9318 | 14.58 |
val: {'recall': 0.994702, 'recall_grapheme': 0.992307, 'recall_vowel': 0.997553, 'recall_consonant': 0.996642, 'recall_word': 0.993668, 'acc_grapheme': 0.993576, 'acc_vowel': 0.997784, 'acc_consonant': 0.997859, 'acc_word': 0.993651, 'loss_grapheme': 0.029794, 'loss_vowel': 0.013136, 'loss_consonant': 0.01068, 'loss_word': 0.02701}
   11 | 0.000159 | 160640/160678 | 0.8778 | 6.0795 | 14.56 ||
val: {'recall': 0.995492, 'recall_grapheme': 0.99389, 'recall_vowel': 0.998107, 'recall_consonant': 0.996083, 'recall_word': 0.994334, 'acc_graph

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: False

val: {'recall': 0.796988, 'recall_grapheme': 0.763845, 'recall_vowel': 0.863414, 'recall_consonant': 0.796848, 'recall_word': 0.762264, 'acc_grapheme': 0.493819, 'acc_vowel': 0.788904, 'acc_consonant': 0.767645, 'acc_word': 0.316245, 'loss_grapheme': 1.992506, 'loss_vowel': 0.626105, 'loss_consonant': 0.522644, 'loss_word': 3.290409}
CYCLE: 1


/home/chec/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    0 | 0.000292 | 160640/160716 | 6.7174 | 10.7151 | 13.99 ||
val: {'recall': 0.992886, 'recall_grapheme': 0.989107, 'recall_vowel': 0.996141, 'recall_consonant': 0.997187, 'recall_word': 0.9882, 'acc_grapheme': 0.989607, 'acc_vowel': 0.996087, 'acc_consonant': 0.996087, 'acc_word': 0.988212, 'loss_grapheme': 0.129831, 'loss_vowel': 0.068911, 'loss_consonant': 0.05198, 'loss_word': 0.144907}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth
    1 | 0.000543 | 160640/160716 | 7.1504 | 7.3232 | 14.15 ||
val: {'recall': 0.992768, 'recall_grapheme': 0.989784, 'recall_vowel': 0.995191, 'recall_consonant': 0.996311, 'recall_word': 0.989335, 'acc_grapheme': 0.99018, 'acc_vowel': 0.995364, 'acc_consonant': 0.995464, 'acc_word': 0.988884, 'loss_grapheme': 0.069323, 'loss_vowel': 0.043396, 'loss_consonant': 0.0354, 'loss_word': 0.072684}
    2 | 0.000476 | 160640/160716 | 16.1799 | 6.6101 | 14.24 |
val: {'recall': 0.994403, 'recall_grapheme': 0.992025, 'recall_vowel': 0.9

KeyboardInterrupt: 

In [44]:
time.time()

1582104595.1300395

In [ ]:
#save_model(model, model_file)

In [ ]:
del model